| 목적      | 검색 쿼리 (EuropePMC용)                                                     | 설명                          |
| ------- | ---------------------------------------------------------------------- | --------------------------- |
| 항생제 오남용 | `("antibiotic resistance" OR "antibiotic overuse") AND PUB_YEAR:2025`  | 감기, 독감에 항생제 오용 이슈           |
| 비타민 효능  | `("vitamin D" OR "vitamin C") AND PUB_YEAR:2025`                       | 면역력 관련한 사회적 통념과 최신 결과 비교    |
| 백신 효과   | `("vaccine efficacy" OR "mRNA vaccine") AND PUB_YEAR:2025`             | COVID 이후 백신 신뢰도 관련 정보       |
| 자가면역질환  | `("autoimmune disease" OR "inflammation") AND PUB_YEAR:2025`           | 루푸스, 크론병 등 만성염증 관련 최신 치료법   |
| 암 치료 변화 | `("cancer therapy" OR "immunotherapy") AND PUB_YEAR:2025`              | 사회적으로 널리 알려진 암 치료법 vs 최신 동향 |
| 커피와 건강  | `("coffee consumption" AND "health") AND PUB_YEAR:2025`                | 커피의 심혈관계·인지기능에 미치는 최신 영향 분석 |
| 식이요법    | `("intermittent fasting" OR "low carb diet") AND PUB_YEAR:2025`        | 다이어트 및 대사 건강 관련 통념 vs 최신 연구 |
| 정신건강    | `("depression treatment" OR "mental health stigma") AND PUB_YEAR:2025` | 약물치료·인지행동치료에 대한 최신 트렌드      |
| 건강보조식품  | `("probiotics" OR "supplements") AND PUB_YEAR:2025`                    | 유산균·영양제에 대한 기대효과 검증         |
| 대체의학    | `("acupuncture" OR "herbal medicine") AND PUB_YEAR:2025`               | 침술, 한약 등의 과학적 근거 비교         |

("antibiotic overuse" OR "vitamin C" OR "mRNA vaccine" OR "inflammation" OR "immunotherapy" OR "low carb diet" OR "mental health stigma" OR "supplements" OR "herbal medicine") AND PUB_YEAR:2025


("vitamin D" OR "vitamin C" OR "vaccine efficacy" OR "mRNA vaccine" OR "autoimmune disease" OR "inflammation" OR "cancer therapy" OR "immunotherapy" OR "coffee consumption" AND "health" OR "intermittent fasting" OR "low carb diet" OR "depression treatment" OR "mental health stigma" OR "probiotics" OR "supplements" OR "acupuncture" OR "herbal medicine")

## 1. 본문 텍스트 정제
현재는 p, h2, h3, li 태그의 get_text(strip=True)를 단순히 이어붙이고 있는데, 다음 보완을 추천합니다:

👉 보완 항목
- 불필요한 공백, 개행 정리 (\n\n\n → \n\n)
- 숫자 목록 (1., 2.), 문단번호 ([1], (1)) 제거
- 테이블 내용, 수식, 그림 캡션(figure, table) 등 제거 또는 별도 저장
- References, Acknowledgement, Funding, Conflict of Interest 등의 말미 섹션 제거 (내용 검색에 방해)

```python
# 예: 불필요한 섹션 제거
if 'references' in section.text.lower(): break
```

## 2. 메타데이터 정제 및 보강
👉 개선 사항
- 현재 저장 중인 메타데이터: source, pmcid, title
- 추가 추천:
    - pubYear: 발행연도
    - journalTitle: 학술지명
    - authorString: 저자 정보
    - doi: DOI (→ 나중에 PDF 크롤링이나 citation 연결에도 유리)
    - abstractText: 요약 정보 (본문 없이 fallback 용도로 활용 가능)

```python
# PMC search 결과에 포함된 필드 활용
pmcid = r.get("pmcid", "")
doi = r.get("doi", "")
pubYear = r.get("pubYear", "")
journal = r.get("journalTitle", "")
abstract = r.get("abstractText", "")
authors = r.get("authorString", "")

```

## 3. 중복 처리 로직 보완
현재 source URL로 중복 여부를 판단하고 있는데, 이는 구조가 변경되면 유효하지 않을 수 있습니다. 아래 보완을 권장합니다:
- 중복 체크 기준을 pmcid 또는 doi 기반으로 고정
- 추후 PDF 다운로드 및 RAG 연결 시에도 pmcid나 doi가 더 확실한 키가 됩니다

```python
existing_pmcs = {metadata.get('pmcid') for metadata in existing_docs.get('metadatas', []) if metadata}
```

## 4. 오류 회피형 예외 처리 강화
일부 논문에서 <div>가 존재하더라도 내용이 누락될 수 있습니다.
```python
if not full_text or len(full_text.strip()) < 200:
    raise Exception("❌ 내용 부족으로 저장 스킵")
```

## 5. RAG 활용성 향상을 위한 태깅
향후 RAG 모델에서 조건 검색(예: “2025년 발표된 암 관련 논문 중 ‘자가면역’ 키워드 포함”)을 하기 위해서는:
- 메타데이터 외에도 chunk 단위에 keyword 태그 추가
- 미리 특정 키워드를 탐지해 tag로 추가하는 것도 고려 가능

예시: 개선된 Document 메타데이터 구성

```python
doc.metadata = {
    "source": source_url,
    "pmcid": pmcid,
    "title": title,
    "doi": doi,
    "pubYear": pub_year,
    "journal": journal,
    "authors": authors,
    "abstract": abstract,
    "tags": ["autoimmune", "cancer", "AI diagnosis"]  # (선택)
}

```

🔍 추가 제안: 향후 PDF 수집 고려 시
PMC HTML이 아닌 PDF 전문을 병행 저장할 경우:
- DOI 기반 EuropePMC PDF 다운로드 경로 구성
- OCR 또는 PDF-Text 추출 파이프라인 추가

✅리팩터링 요약
- 텍스트 정제: 불필요한 섹션 제거(references, funding 등), 본문 최소 길이 조건 추가
- 메타데이터 확장: DOI, 저널명, 발행년도, 저자, 초록 등 추가 → RAG 기반 LLM에서 질의 가능성 ↑
- 중복 처리 강화: pmcid 기반으로 중복 제거
- 코드 정리: 함수 구조화 및 각 처리 단계 시간 최소화

## 1. 본문 HTML 구조가 다름 (추출 실패)

extract_pmc_fulltext() 함수에서는 다음 영역에서 본문을 찾습니다:

```python
soup.select("div.tsec, div.sec, div.body, div.main, div#article-text, article")
```

하지만 일부 논문은 위의 영역들이 존재하지 않거나, <p>, <h2> 같은 태그가 존재하더라도 본문이 비어 있거나 매우 짧은 경우가 있습니다.

```python
if len(full_text.strip()) < 100:
    raise Exception(f"❌ 목록이 널리는 내용: {url}")
```

즉, 이 메시지는:
> 본문 추출은 되었지만, 내용이 너무 짧아서 의미 없는 논문일 가능성이 높다는 의미입니다.